In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
cd /content/drive/MyDrive/DLproject/DLproject

/content/drive/MyDrive/DLproject/DLproject


In [9]:
!pip install ipywidgets==7.7.1
!git config --global user.email 'sus5467@naver.com'
!git config --global user.name 'xuio-0528'

In [15]:
import huggingface_hub

In [16]:
huggingface_hub.login()

In [22]:
! git add .

fatal: not a git repository (or any of the parent directories): .git


In [23]:
! git commit -m "llama-7b 돌아가는 것 확인"

fatal: not a git repository (or any of the parent directories): .git


In [21]:
! git push

fatal: not a git repository (or any of the parent directories): .git


## 개요

어떤 유형의 프로젝트를 진행할 것인가?
### 후보군
1. prompt tuning
2. LoRA
3. small model 학습

### 장단점
#### 1. prompt tuning
- prompt tuning의 경우 오랜 학습이 필요하지 않음
- 어떤 부분을 차용해서 해볼 것인지에 대한 고려가 필요하다.

#### 2. LoRA
- 오랜 학습이 필요하지는 않음
- LoRA로 무엇을 학습시킬 것인지 고안하는 것이 중요
- 학습이 잘 되지 않을 수 있다는 위험성이 존재


#### 3. small model 학습
- 성능 상승이 거의 일어나기 어렵다.
- 별로 매력적이지 않음

#### 번외
- 1번 2번 두 가지 모두 이용을 해보는 것도 좋은 선택지가 될 것 같다.

### 다룰 모델들은?
1. 코드 생성 모델
2. reasoning model

위의 두 모델 같은 경우에는 아무래도 정확도가 어느 정도 담보가 되어 있는 모델들이 좋다.

코드 생성 모델의 경우 phi-1.5를 사용할 것 같다.
reasoning model의 경우 llama를 사용하는 것이 좋을 것 같다.

그렇다면 LoRA로 무엇을 가르쳐볼까?

일반적으로 우리는 prompt를 학습하거나 freeze시키는 등을 통해 embedding에서의 수정을 해보고자 한다.

혹은 하나의 prompt만을 동일하게 사용하려는 것을 보인다.
예를 들어, let's think about step by step 등

그렇다면 prompt를 동일하게 사용하는 것이 아니라 prompt를 적절하게 generate를 시켜줄 수는 없을까?

text description을 요약 모델에 넣어주어 title 혹은 summary를 요약해준다.

summary와 원래의 text description을 모델에 동시에 제공하고 그것을 토대로 학습.

그렇다면 GAN처럼 원래의 loss와 prompt를 넣었을 때의 loss를 비교하고 loss가 최소화되는 방향으로 prompt를 regenerate시키는 방향..?

우선 llama-7b1와 phi-1.5b가 inference는 되는지 확인

In [5]:
!pip install transformers
!pip install datasets
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


In [18]:
import os
import sys

import numpy
import pandas as pd
import torch
import transformers

import json

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# print(data)

torch.set_default_device("cuda")
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
inputs = tokenizer('''```python
    def print_prime(n):
    """
    Print all primes between 1 and n
    """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

OutOfMemoryError: ignored

llama-7b는 T4로는 load 자체도 터져버린다.

그렇다면 유료로 결제하는 방식을 사용해야 하나

phi-1.5b는 되는 것을 확인
즉, prompt tuning은 가능할 것

In [ ]:
import torch
from torch.utils.data import Dataset
import json
from datasets import load_dataset

class CodeDataset(Dataset):
    def __init__(self, dataset_name, data_type):
        self.examples = []
        self.data_type = data_type
        self.data = load_dataset(dataset_name)[self.data_type]

    def map(self, func):
        self.data = [func(item) for item in self.data]
        

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):

        return self.data[idx]

In [ ]:
import json
import os.path as osp
from typing import Union


class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = osp.join("templates", f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: Union[None, str] = None,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if not instruction and input:
            res = self.template["prompt_no_instruction"].format(input=input)
        elif input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        # if input:
        #     res = self.template["prompt_input"].format(
        #         input=input
        #     )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1]
    
    def get_input(self, output: str) -> str:
        return output.split(self.template["response_split"])[0]

In [ ]:
import os
import sys
from typing import List

import numpy as np
import fire
import torch
import transformers
from datasets import load_dataset
from code_dataset import CodeDataset

"""
Unused imports:
import torch.nn as nn
import bitsandbytes as bnb
"""

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoTokenizer, CodeLlamaTokenizer
from transformers import TrainerCallback, get_cosine_schedule_with_warmup

from utils.prompter import Prompter


# class LogCallback(TrainerCallback):
#     def init(self, state):
#         self.state = state
#     def on_step_end(self, args, state, control, logs = None, logging_steps=1, **kwargs):
#         if state.global_step % logging_steps == 0:
#             # 매 logging_steps 스텝마다 training loss 출력
#             print(f"Step {state.global_step}: Train loss {state.log_history[-1]}")

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def train(
    # model/data params
    base_model: str = "microsoft/phi-1_5",  # the only required argument
    output_dir: str = "./",
    # training hyperparams
    batch_size: int = 1,
    micro_batch_size: int = 1,
    num_epochs: int = 200,
    learning_rate: float = 3e-3,
    cutoff_len: int = 1024,
    val_set_size: int = 2000,
    # lora hyperparams
    lora_r: int = 8,
    lora_alpha: int = 16,
    lora_dropout: float = 0.05,
    # lora_target_modules: List[str] = [
    #     "q_proj",
    #     "v_proj",
    # ],
    lora_target_modules: List[str] = ['q_proj','k_proj','v_proj','o_proj'],
    # llm hyperparams
    train_on_inputs: bool = True,  # if False, masks out inputs in loss
    add_eos_token: bool = True,
    group_by_length: bool = False,  # faster, but produces an odd training loss curve
    # wandb params
    wandb_project: str = "lora-llama2-code",
    wandb_run_name: str = "1data50epoch",
    wandb_watch: str = "false",  # options: false | gradients | all
    wandb_log_model: str = "llama2",  # options: false | true
    resume_from_checkpoint: str = None,  # either training checkpoint or final adapter
    prompt_template_name: str = "no_instruction",  # The prompt template to use, will default to alpaca.
    seed: int = 42,
):
    if int(os.environ.get("LOCAL_RANK", 0)) == 0:
        print(
            f"Training Alpaca-LoRA model with params:\n"
            f"base_model: {base_model}\n"
            f"output_dir: {output_dir}\n"
            f"batch_size: {batch_size}\n"
            f"micro_batch_size: {micro_batch_size}\n"
            f"num_epochs: {num_epochs}\n"
            f"learning_rate: {learning_rate}\n"
            f"cutoff_len: {cutoff_len}\n"
            f"val_set_size: {val_set_size}\n"
            f"lora_r: {lora_r}\n"
            f"lora_alpha: {lora_alpha}\n"
            f"lora_dropout: {lora_dropout}\n"
            f"lora_target_modules: {lora_target_modules}\n"
            f"train_on_inputs: {train_on_inputs}\n"
            f"add_eos_token: {add_eos_token}\n"
            f"group_by_length: {group_by_length}\n"
            f"wandb_project: {wandb_project}\n"
            f"wandb_run_name: {wandb_run_name}\n"
            f"wandb_watch: {wandb_watch}\n"
            f"wandb_log_model: {wandb_log_model}\n"
            f"resume_from_checkpoint: {resume_from_checkpoint or False}\n"
            f"prompt template: {prompt_template_name}\n"
        )
    assert (
        base_model
    ), "Please specify a --base_model, e.g. --base_model='huggyllama/llama-7b'"
    gradient_accumulation_steps = batch_size // micro_batch_size
    # seed 설정
    set_seed(seed)
    
    prompter = Prompter(prompt_template_name)

    device_map = "auto"
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    ddp = world_size != 1
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
        gradient_accumulation_steps = gradient_accumulation_steps // world_size

    # Check if parameter passed or if set within environ
    use_wandb = len(wandb_project) > 0 or (
        "WANDB_PROJECT" in os.environ and len(os.environ["WANDB_PROJECT"]) > 0
    )
    # Only overwrite environ if wandb param passed
    if len(wandb_project) > 0:
        os.environ["WANDB_PROJECT"] = wandb_project
    if len(wandb_watch) > 0:
        os.environ["WANDB_WATCH"] = wandb_watch
    if len(wandb_log_model) > 0:
        os.environ["WANDB_LOG_MODEL"] = wandb_log_model

    model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", 
                                                 load_in_8bit=True,
                                                 torch_dtype=torch.float16,
                                                 device_map=device_map,
                                                 trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
    
    tokenizer.padding_side = "left"
    tokenizer.pad_token_id = (
        0  
    )
    
    def tokenize(prompt, add_eos_token=True):
        result = tokenizer(
            prompt,
            truncation=True,
            max_length=cutoff_len,
            padding="max_length",
            return_tensors=None,
        )
        if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < cutoff_len
            and add_eos_token
        ):
            result["input_ids"].append(tokenizer.eos_token_id)
            result["attention_mask"].append(1)

        result["labels"] = result["input_ids"].copy()

        return result

    def generate_and_tokenize_prompt(data_point):
        full_prompt = prompter.generate_prompt(
            input = data_point["input"],
            label = data_point["output"],
        )
        tokenized_full_prompt = tokenize(full_prompt, add_eos_token=add_eos_token)
        if not train_on_inputs:
            user_prompt = prompter.generate_prompt(
                data_point["input"]
            )
            tokenized_user_prompt = tokenize(
                user_prompt, add_eos_token=add_eos_token
            )
            user_prompt_len = len(tokenized_user_prompt["input_ids"])

            if add_eos_token:
                user_prompt_len -= 1

            tokenized_full_prompt["labels"] = [
                -100
            ] * user_prompt_len + tokenized_full_prompt["labels"][
                user_prompt_len:
            ]  # could be sped up, probably
        return tokenized_full_prompt

    model = prepare_model_for_int8_training(model)
    # model.resize_token_embeddings(len(tokenizer))
    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model.enable_input_require_grads()
    model = get_peft_model(model, config)

    train_data = CodeDataset("codeparrot/apps", "train")
    val_data = CodeDataset("codeparrot/apps", "test")

    train_data.map(generate_and_tokenize_prompt)
    val_data.map(generate_and_tokenize_prompt)
    
    if resume_from_checkpoint:
        # Check the available weights and load them
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "pytorch_model.bin"
        )  # Full checkpoint
        if not os.path.exists(checkpoint_name):
            checkpoint_name = os.path.join(
                resume_from_checkpoint, "adapter_model.bin"
            )  # only LoRA model - LoRA config above has to fit
            resume_from_checkpoint = (
                False  # So the trainer won't try loading its state
            )
        # The two files above have a different name depending on how they were saved, but are actually the same.
        if os.path.exists(checkpoint_name):
            print(f"Restarting from {checkpoint_name}")
            adapters_weights = torch.load(checkpoint_name)
            set_peft_model_state_dict(model, adapters_weights)
        else:
            print(f"Checkpoint {checkpoint_name} not found")

    model.print_trainable_parameters()  # Be more transparent about the % of trainable params.
    
    if not ddp and torch.cuda.device_count() > 1:
        model.is_parallelizable = True
        model.model_parallel = True
    
    
    trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=micro_batch_size,
            per_device_eval_batch_size=micro_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            warmup_steps=0,
            weight_decay=0.01, ## 새롭게 설정
            num_train_epochs=num_epochs,
            learning_rate=learning_rate,
            fp16=True,
            logging_steps=1,
            logging_dir='./logs',
            optim="adamw_torch",
            lr_scheduler_type = 'cosine', 
            evaluation_strategy="steps" if val_set_size > 0 else "no",
            save_strategy="steps",
            eval_steps=50 if val_set_size > 0 else None,
            save_steps=50,
            output_dir=output_dir,
            save_total_limit=3,
            load_best_model_at_end=True if val_set_size > 0 else False,
            ddp_find_unused_parameters=False if ddp else None,
            group_by_length=group_by_length,
            report_to="wandb" if use_wandb else None,
            run_name=wandb_run_name if use_wandb else None,
        ),
        data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=False
        )
    )
    model.config.use_cache = False

    old_state_dict = model.state_dict
    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(
            self, old_state_dict()
        )
    ).__get__(model, type(model))

    if torch.__version__ >= "2" and sys.platform != "win32":
        model = torch.compile(model)

    trainer.train(resume_from_checkpoint=resume_from_checkpoint)

    model.save_pretrained(output_dir)

    print(
        "\n If there's a warning about missing keys above, please disregard :)"
    )


if __name__ == "__main__":
    fire.Fire(train)